In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections

In [2]:
def collect_losses(root_dir):

    depth = 2

    seeds = [123, 456, 789]

    dict_res = dict() # {"dataset_name" : [means stats, var stats]}

    # collect ts losses

    for seed_ in seeds:
        seed_rootdir = f"{root_dir}/{seed_}"

        for subdir, dirs, files in os.walk(seed_rootdir):
            if subdir[len(seed_rootdir):].count(os.sep) < depth:
                for file_ in files:
                    if file_ == "finetuning_results.json":
                        with open(subdir + "/" + file_) as f:
                            d = json.load(f)
                            losses_ts = list(d["losses_ts"].values())

                        dataset_name = subdir.split(os.sep)[-1]
                        if dataset_name in dict_res.keys():
                            dict_res[dataset_name].append(losses_ts)
                        else:
                            dict_res[dataset_name] = [losses_ts]

    # compute mean and std for each losses

    for dataset, losses_list in dict_res.items():
        means = []
        stds = []

        losses_len = len(losses_list[0])
        for i in range(losses_len):
            losses_i = []
            for losses in losses_list:
                losses_i.append(losses[i])
            means.append(np.mean(losses_i))
            stds.append(np.std(losses_i))

        dict_res[dataset] = [means, stds]

    return dict_res


def avg_losses_over_datasets(datasets_losses):
    avg_losses_dts = None

    num_dts = 0

    for dataset, mean_std in datasets_losses.items():
        means = mean_std[0]

        if avg_losses_dts is None:
            avg_losses_dts = np.array(means)
        else:
            avg_losses_dts += np.array(means)

        num_dts += 1

    return avg_losses_dts / num_dts


## Finetuning Results

### Not Pretraining

In [3]:
res_ = collect_losses("../finetuning/notpretraining")

res = dict()

for k, v in res_.items():
    res[" ".join(k.split("_")[-3:])] = v[0]
    
res = collections.OrderedDict(sorted(res.items()))

In [4]:
pd.DataFrame.from_dict(res).T

,0,1,2,3,4,5,6,7,8,9
all other rand,103.172704,15.511683,21.041946,34.421854,11.828411,8.409997,13.985721,9.982960,10.428221,13.994352
en all notpretraining,102.986341,15.476217,21.005109,34.441061,11.841845,8.296990,13.936183,9.817902,10.455311,14.015338
ge 0 notpretraining,164.064482,14.831715,16.739370,18.998782,7.446870,3.650347,7.808324,4.559206,26.529394,32.921420
ge 1 notpretraining,94.633396,12.487382,14.595778,21.982324,6.866500,6.386626,8.121793,7.299536,19.562414,15.084082
ge 2 notpretraining,98.751697,13.191884,19.601118,18.199411,8.608217,8.482743,9.867734,8.789848,15.507332,16.478669
ge all notpretraining,106.017858,13.599363,14.786019,27.619094,7.945404,7.725458,9.344697,8.934400,16.813270,15.626563
it 0 notpretraining,228.685047,18.058541,21.211203,21.685961,9.345869,8.188477,12.176523,5.878339,28.497592,37.484364
it 1 notpretraining,236.298087,15.375296,40.511067,14.258837,9.601842,7.168391,14.017693,7.665295,9.152631,20.626607
it 2 notpretraining,142.301854,14.002409,21.048478,27.530566,7.104128,4.994385,7.965759,7.565304,18.684336,17.126319
it all notpretraining,147.912665,14.998925,20.075226,30.502404,9.381310,6.301812,10.171060,7.735195,17.371903,18.452492


### Pretraining

In [5]:
res_ = collect_losses("../finetuning/pretraining")

res = dict()

for k,v in res_.items():
    res[" ".join(k.split("_")[-3:])] = v[0]
    
res = collections.OrderedDict(sorted(res.items()))

In [6]:
pd.DataFrame.from_dict(res).T

,0,1,2,3,4,5,6,7,8,9
en all pretraining,120.037827,16.983292,22.848126,37.899501,13.271264,9.309837,15.718107,10.956735,10.778671,15.084091
ge 0 pretraining,176.095910,15.166723,15.678791,21.179621,8.148184,3.629168,8.760403,4.655479,25.774425,33.507715
ge 1 pretraining,107.414960,13.549938,14.781487,25.391632,7.816271,7.162209,9.526576,8.243808,19.878175,15.599346
ge 2 pretraining,110.802051,14.157869,19.909969,20.755117,9.663459,9.394652,11.303693,9.768973,15.482056,16.985038
ge all pretraining,122.604580,14.908806,15.266185,31.330749,9.111731,8.776475,10.964026,10.138844,17.142778,16.539663
it 0 pretraining,249.763954,18.704402,20.357604,23.829548,10.149569,8.704612,13.779179,6.194905,28.247145,38.372658
it 1 pretraining,245.168348,15.164032,39.696565,14.344321,9.530321,7.000390,14.039581,7.493192,9.012794,20.195095
it 2 pretraining,158.567245,14.830383,21.582696,30.325948,7.755447,5.408523,8.909906,8.194049,18.902115,17.564381
it all pretraining,168.290214,16.169623,20.951977,33.670970,10.372961,6.902787,11.471832,8.448477,17.895924,19.642061
sp 0 pretraining,197.684468,19.861150,24.410014,26.188791,17.213783,13.899291,21.912076,17.250822,14.905767,23.108660


### Not Pretraining Not Full

In [7]:
res_ = collect_losses("../finetuning/notpretraining_notfull")

res = dict()

for k,v in res_.items():
    res[" ".join(k.split("_")[-4:])] = v[0]
    
res = collections.OrderedDict(sorted(res.items()))

In [8]:
pd.DataFrame.from_dict(res).T

,0,1,2,3,4,5,6,7,8,9
en all notpretraining notfull,164.634745,20.456924,27.850174,45.296115,16.521255,11.415674,19.205664,13.361401,12.232790,17.772320
ge 0 notpretraining notfull,213.619452,16.891593,16.321663,26.314154,10.054450,4.204023,10.867317,5.421867,27.181063,34.768214
ge 1 notpretraining notfull,149.354657,16.638355,17.472611,33.001053,10.601880,9.396280,12.644981,10.819846,22.275451,16.894736
ge 2 notpretraining notfull,151.622826,17.070775,23.243080,26.559485,12.675416,11.897022,14.643554,12.420344,16.685903,18.441399
ge all notpretraining notfull,171.306494,18.408007,18.347836,39.348834,12.140155,11.302330,14.249024,13.006752,19.411457,19.457668
it 0 notpretraining notfull,307.014537,20.685821,20.995719,28.285731,12.450887,10.234952,16.662091,7.317388,28.642855,40.896944
it 1 notpretraining notfull,282.072150,16.212598,41.403742,16.276469,10.997414,8.061635,16.035754,8.622203,8.760318,19.543251
it 2 notpretraining notfull,212.358820,17.525890,24.761929,36.979123,9.951669,6.891594,11.105098,10.190315,20.992685,19.334710
it all notpretraining notfull,227.249141,19.283068,24.540554,40.561953,13.060082,8.618962,14.160312,10.446942,20.192612,22.683125
sp 0 notpretraining notfull,256.740125,23.049310,28.479493,30.829698,20.678415,16.389662,26.015629,20.370343,16.094657,25.536591


### Pretraining Not Full

In [9]:
res_ = collect_losses("../finetuning/pretraining_notfull")

res = dict()

for k,v in res_.items():
    res[" ".join(k.split("_")[-4:])] = v[0]
    
res = collections.OrderedDict(sorted(res.items()))

In [10]:
pd.DataFrame.from_dict(res).T

,0,1,2,3,4,5,6,7,8,9
en all pretraining notfull,194.058971,23.165393,30.488639,48.651314,19.268808,14.112768,22.351495,16.546084,13.776630,20.127411
ge 0 pretraining notfull,244.373958,18.328586,14.562329,29.772989,12.853234,5.984503,14.048438,7.994384,26.318477,35.094336
ge 1 pretraining notfull,181.785675,18.996994,17.859540,36.818724,13.671184,12.426007,16.106836,14.330118,23.039999,17.723549
ge 2 pretraining notfull,184.056343,19.197285,23.703133,30.221692,15.631368,14.797049,18.022663,15.780549,16.292656,19.129172
ge all pretraining notfull,208.066608,21.159537,19.510757,43.174040,15.261447,14.417805,17.751767,16.595695,20.801662,21.763120
it 0 pretraining notfull,343.993289,22.146022,20.058960,31.135279,14.733255,12.391185,19.335079,9.671823,28.207753,41.634844
it 1 pretraining notfull,306.844717,16.030924,40.860007,16.909834,11.511856,8.559508,16.674317,9.185393,6.616343,17.930128
it 2 pretraining notfull,244.716206,19.219086,24.896585,40.024774,12.184595,8.865793,13.764764,12.792940,21.262528,19.960710
it all pretraining notfull,263.775253,21.416216,25.458905,43.791590,15.568774,10.869399,17.061465,13.225536,21.077258,24.276797
sp 0 pretraining notfull,296.349261,25.066845,29.371691,33.959941,23.233404,18.905636,29.026201,23.379820,16.069914,26.588159


### Not Pretraining Dur

In [11]:
res_ = collect_losses("../finetuning/notpretraining_dur")

res = dict()

for k,v in res_.items():
    res[" ".join(k.split("_")[-4:])] = v[0]
    
res = collections.OrderedDict(sorted(res.items()))

In [12]:
pd.DataFrame.from_dict(res).T

,0,1,2
en all notpretraining dur,15.426181,5.342428,5.342428
ge 0 notpretraining dur,12.145139,3.842114,3.842114
ge 1 notpretraining dur,16.459385,5.037882,5.037882
ge 2 notpretraining dur,26.432882,6.683137,6.683137
ge all notpretraining dur,19.295539,5.493298,5.493298
it 0 notpretraining dur,33.200928,7.099625,7.099625
it 1 notpretraining dur,34.148488,7.227250,7.227250
it 2 notpretraining dur,12.588708,3.998771,3.998771
it all notpretraining dur,14.883593,4.467051,4.467051
sp 0 notpretraining dur,51.158353,8.997917,8.997917


### Pretraining Dur

In [13]:
res_ = collect_losses("../finetuning/pretraining_dur")

res = dict()

for k,v in res_.items():
    res[" ".join(k.split("_")[-4:])] = v[0]
    
res = collections.OrderedDict(sorted(res.items()))

In [14]:
pd.DataFrame.from_dict(res).T

,0,1,2
en all pretraining dur,15.831568,5.000624,5.000624
ge 0 pretraining dur,9.686704,3.298628,3.298628
ge 1 pretraining dur,14.558344,4.277087,4.277087
ge 2 pretraining dur,26.036299,6.264882,6.264882
ge all pretraining dur,18.854497,5.020077,5.020077
it 0 pretraining dur,31.168761,6.506721,6.506721
it 1 pretraining dur,27.944759,6.152115,6.152115
it 2 pretraining dur,9.670235,3.147800,3.147800
it all pretraining dur,12.209374,3.752118,3.752118
sp 0 pretraining dur,62.326054,9.849493,9.849493


### Not Pretraining Prob Skip

In [15]:
res_ = collect_losses("../finetuning/notpretraining_prob_skip")

res = dict()

for k,v in res_.items():
    res[" ".join(k.split("_")[-5:])] = v[0]
    
res = collections.OrderedDict(sorted(res.items()))

In [16]:
pd.DataFrame.from_dict(res).T

,0,1,2
en all notpretraining prob skip,147.232560,20.176378,20.176378
ge 0 notpretraining prob skip,200.628741,15.264225,15.264225
ge 1 notpretraining prob skip,107.005020,13.490433,13.490433
ge 2 notpretraining prob skip,199.422566,22.395260,22.395260
ge all notpretraining prob skip,102.875232,13.810207,13.810207
it 0 notpretraining prob skip,270.894147,19.404060,19.404060
it 1 notpretraining prob skip,861.430774,44.572635,44.572635
it 2 notpretraining prob skip,302.361153,25.234571,25.234571
it all notpretraining prob skip,248.641138,22.842001,22.842001
sp 0 notpretraining prob skip,267.767778,25.479436,25.479436


### Pretraining Prob Skip

In [17]:
res_ = collect_losses("../finetuning/pretraining_prob_skip")

res = dict()

for k,v in res_.items():
    res[" ".join(k.split("_")[-5:])] = v[0]
    
res = collections.OrderedDict(sorted(res.items()))

In [18]:
pd.DataFrame.from_dict(res).T

,0,1,2
en all pretraining prob skip,175.224570,20.590183,20.590183
ge 0 pretraining prob skip,199.044119,14.574144,14.574144
ge 1 pretraining prob skip,110.393690,12.696182,12.696182
ge 2 pretraining prob skip,199.984169,17.676861,17.676861
ge all pretraining prob skip,107.625747,12.957890,12.957890
it 0 pretraining prob skip,285.881290,18.402621,18.402621
it 1 pretraining prob skip,976.586255,39.172789,39.172789
it 2 pretraining prob skip,306.157414,20.095074,20.095074
it all pretraining prob skip,259.892711,19.584855,19.584855
sp 0 pretraining prob skip,288.866269,22.486225,22.486225
